# Módulo 6: Big data con pyspark - Ejercicio de evaluación

In [42]:
# Importación de librerías necesarias
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, StringType
from pyspark.ml.feature import StringIndexer, Imputer, OneHotEncoder, VectorAssembler, MinMaxScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## 1. Carga de datos de diamonds desde CSV con schema:

- Primero, se descarga el dataset desde un enlace en línea y se guarda localmente.

- Luego, se define un esquema para las columnas de datos (especificando los tipos de datos como DoubleType o StringType).

In [43]:
# Cargar el dataset desde la URL y guardarlo en CSV localmente
dataset_url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv'
csv_path = 'diamonds.csv'
df_pandas = pd.read_csv(dataset_url)
df_pandas.to_csv(csv_path, index=False)

spark = SparkSession.builder.appName("DiamondsPipeline").getOrCreate()

schema = StructType([
    StructField("carat", DoubleType(), True),
    StructField("cut", StringType(), True),
    StructField("color", StringType(), True),
    StructField("clarity", StringType(), True),
    StructField("depth", DoubleType(), True),
    StructField("table", DoubleType(), True),
    StructField("price", DoubleType(), True),
    StructField("x", DoubleType(), True),
    StructField("y", DoubleType(), True),
    StructField("z", DoubleType(), True)
])

df = spark.read.csv(csv_path, header=True, schema=schema)
df = df.cache()  # Cache para optimizar la ejecución

## 2. Pipeline de Regresión para predecir 'price'

- Utilizamos un conjunto de herramientas de preprocesamiento (como Imputer para valores faltantes, StringIndexer para convertir variables categóricas en números, y OneHotEncoder para crear variables binarias) para preparar los datos.

- Escalamos algunas de las características (como carat) utilizando MinMaxScaler.

- Creamos un modelo de regresión con RandomForestRegressor para predecir el precio de los diamantes.

In [48]:
# Imputer para columnas numéricas
imputer_reg = Imputer(
    inputCols=["carat", "depth", "table", "x", "y", "z"],
    outputCols=["carat_imputed", "depth_imputed", "table_imputed", "x_imputed", "y_imputed", "z_imputed"]
)

# Indexar variables categóricas: "cut", "color", "clarity"
indexers = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in ["cut", "color", "clarity"]]

# One-Hot Encoding para las variables categóricas indexadas
encoders = [OneHotEncoder(inputCol=col + "_index", outputCol=col + "_ohe") for col in ["cut", "color", "clarity"]]

# Escalado de la variable "carat" (usando la columna imputada)
carat_assembler = VectorAssembler(inputCols=["carat_imputed"], outputCol="carat_vector")
scaler = MinMaxScaler(inputCol="carat_vector", outputCol="carat_scaled")

# VectorAssembler para combinar las features finales (usando columnas imputadas)
assembler = VectorAssembler(
    inputCols=["carat_scaled", "depth_imputed", "table_imputed", "x_imputed", "y_imputed", "z_imputed", "cut_ohe", "color_ohe", "clarity_ohe"],
    outputCol="features"
)

# Modelo de regresión
regressor = RandomForestRegressor(featuresCol="features", labelCol="price")

# Definición del pipeline de regresión con imputer
reg_pipeline = Pipeline(stages=[imputer_reg] + indexers + encoders + [carat_assembler, scaler, assembler, regressor])

# Entrenamiento y predicción del modelo de regresión
reg_model = reg_pipeline.fit(df)
df_pred_reg = reg_model.transform(df)

# Evaluación del modelo de regresión
evaluator_reg = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator_reg.evaluate(df_pred_reg)
print(f"Regresión R2: {r2}")

evaluator_rmse = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(df_pred_reg)
print(f"Regresión RMSE: {rmse}")

evaluator_mae = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(df_pred_reg)
print(f"Regresión MAE: {mae}")

Regresión R2: 0.906992235373207
Regresión RMSE: 1216.655116436566
Regresión MAE: 685.959520354088


## 3. Pipeline de Clasificación para predecir 'cut'

- Similar al pipeline de regresión, pero en lugar de predecir el precio, predecimos la categoría cut de los diamantes.

- Usamos un modelo de clasificación RandomForestClassifier para este objetivo.

In [49]:
# Indexar "cut" para usarlo como label
label_indexer = StringIndexer(inputCol="cut", outputCol="label")

# Imputer para columnas numéricas
imputer_clf = Imputer(
    inputCols=["carat", "depth", "table", "x", "y", "z"],
    outputCols=["carat_imputed", "depth_imputed", "table_imputed", "x_imputed", "y_imputed", "z_imputed"]
)

# Para clasificación, se usan las variables "color" y "clarity" como features categóricas
indexers_clf = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in ["color", "clarity"]]
encoders_clf = [OneHotEncoder(inputCol=col + "_index", outputCol=col + "_ohe") for col in ["color", "clarity"]]

# Reutilizamos el escalado de "carat" (usando la versión imputada)
carat_assembler_clf = VectorAssembler(inputCols=["carat_imputed"], outputCol="carat_vector")
scaler_clf = MinMaxScaler(inputCol="carat_vector", outputCol="carat_scaled")

# VectorAssembler para el pipeline de clasificación (usando columnas imputadas)
assembler_clf = VectorAssembler(
    inputCols=["carat_scaled", "depth_imputed", "table_imputed", "x_imputed", "y_imputed", "z_imputed", "color_ohe", "clarity_ohe"],
    outputCol="features"
)

# Modelo de clasificación
classifier = RandomForestClassifier(featuresCol="features", labelCol="label")

# Definición del pipeline de clasificación con imputer
clf_pipeline = Pipeline(stages=[label_indexer, imputer_clf] + indexers_clf + encoders_clf + [carat_assembler_clf, scaler_clf, assembler_clf, classifier])

# Entrenamiento y predicción del modelo de clasificación
clf_model = clf_pipeline.fit(df)
df_pred_clf = clf_model.transform(df)

# Evaluación del modelo de clasificación
evaluator_clf = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_clf.evaluate(df_pred_clf)
print(f"Clasificación Accuracy: {accuracy}")

Clasificación Accuracy: 0.6980163144234335


## 4. GridSearch con CrossValidation

- Se realiza un ajuste de parámetros (en este caso, el número de árboles del modelo) utilizando validación cruzada (CrossValidator).

- Después de la validación cruzada, evaluamos el modelo usando el valor R2 para regresión

In [50]:
paramGrid = ParamGridBuilder().addGrid(regressor.numTrees, [1, 20]).build()
crossval = CrossValidator(
    estimator=reg_pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator_reg,
    numFolds=3
)
cv_model = crossval.fit(df)
cv_r2 = evaluator_reg.evaluate(cv_model.transform(df))
print(f"Cross-Validation R2: {cv_r2}")

Cross-Validation R2: 0.906992235373207
